# Physics-informed machine learning for the COVID-19 pandemic: <br>Adherence to social distancing and short-term predictions for eight countries

### G. D. Barmparis and G. P. Tsironis

Preprint: https://arxiv.org/abs/2008.08162v1

# Import needed packages

In [ ]:
# Import needed modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from datetime import datetime, timedelta

# Set the font size to 18pt for all the plots.
import matplotlib
matplotlib.rcParams.update({'font.size': 18})

# Machine Learning packages
import tensorflow as tf
tf.keras.backend.set_floatx('float64')

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Defining the parameters.

In [ ]:
# Countries. (SWt0 : First day for the second wave, since the first case in each country.) 
countries = {        "Greece" : {"SWt0" : 97},
             "United States"  : {"SWt0" : 127},
             "United Kingdom" : {"SWt0" : 134},
                    "Germany" : {"SWt0" : 160},
                     "France" : {"SWt0" : 130},
                "Netherlands" : {"SWt0" : 126},
                      "Italy" : {"SWt0" : 138},
                      "Spain" : {"SWt0" : 140}
            }

runOnly = ["Greece"] # Use 'All' to run them all!
if "All" in runOnly : runOnly = list(countries.keys())
  
# Kind of data
kind = "new_cases" # or new_deaths

# Number of days for short term predictions.
nOfShortPredDays = 7 

# Number of days for long term predictions.
nOfLongPredDays  = 4*7

# Number of days for smoothing plus minus the given date. Average on 2*nOfDaysSmoothing + 1 days.
nOfDaysSmoothing = 3

# Number of grid-point for the SIR model.
nOfSIRPoints = 500

# Use pre-trained model.
loadModel = True

# Patience before early stopping.
patience = 50

# Display progress every "displayStep" epochs.
displayStep = 50

# Total possible number of epochs.
epochs = 50000

# Read data from https://covid.ourworldindata.org

In [ ]:
url = "https://covid.ourworldindata.org/data/owid-covid-data.csv"
data = pd.read_csv(url)

# Define the Loss functions.

In [ ]:
# Data driven mean squeared error.
def loss_MSE(y_true, y_pred) :
  #
  lossMSE = tf.reduce_mean(tf.square(y_true - y_pred))
  #
  return lossMSE

# PINN-SIR loss.
def sir(model, xTrainSIR, mu):
  #
  tTensor = tf.convert_to_tensor(xTrainSIR)
  tModelA = xTrainSIR.flatten()
  #
  with tf.GradientTape() as g:
    g.watch(tTensor)
    with tf.GradientTape() as gg:
      gg.watch(tTensor)
      x = model(tTensor)
    x_t = gg.gradient(x, tTensor)
    #
  x_tt = g.gradient(x_t, tTensor)
  #
  aa  = sigma*tModelA + sigma0
  a_t = [sigma]*len(tModelA)
  #
  del g, gg
  #
  return tf.abs(x_tt + ((sigma*tModelA + sigma0)*tf.exp(x) - sigma/(sigma*tModelA + sigma0))*(x_t + mu))**2


def loss_SIR() :
  #
  lossSIR = sir(model, xTrainSIR, mu)
  #
  return lossSIR

# Define an Early Stopping class.

In [ ]:
class EarlyStopping :
  #
  def __init__(self, model, patience, displayStep, realData, trainData, tPredict) :
    #
    self.step = 0
    #
    self.fn = open(os.path.join("SecondWave_%s" % country, "%s_output" % country), "a+")
    #
    self.model = model
    #
    self.patience = patience
    #
    self.displayStep = displayStep
    self.tData, self.yData = realData
    self.tTrainData, self.yTrainData = trainData
    self.tPredict = tPredict
    #
    self.best = np.inf
    self.wait = 0
    #
    self.best_model = None
    self.best_weights = None
    self.bestSigma = None
    self.bestSigma0 = None
    self.bestM = None
    #
    self.stopped_epoch = 0
    self.stop_training = False

  def on_epoch_end(self, epoch, losses) :
    #
    mse, pi, current = losses["MSE_Data"], losses["MSE_SIR"], losses["TotLoss"]
    #
    if current < 1.e-5 :
      #
      self.stopped_epoch = epoch
      #
      self.model.set_weights(self.best_weights)
      self.model.save(os.path.join("SecondWave_%s" % country, "%s_SIR.h5" % country))
      #
      log = "Epoch %05d: early stopping. %s" % (self.stopped_epoch + 1, " ".join("%s : %12.6f " % (k,v) for k,v in losses.items()))
      #
      print(log)
      #
      self.fn.write("%s\n" % log)
      #
      self.plotBestState(epoch)
      #
      self.plotPredictions()
      #
      self.stop_training = True
      #
    else :
      #
      if np.less(current, self.best) :
        #
        self.best = current
        self.wait = 0
        #
        # Keep best weights.
        self.best_model = self.model
        self.best_weights = self.model.get_weights()
        self.bestSigma = sigma.numpy()
        self.bestSigma0 = sigma0.numpy()
        self.bestM = mu.numpy()
        #
      else :
        #
        self.wait += 1
        #
        if self.wait >= self.patience:
          #
          self.stopped_epoch = epoch
          #
          log = "Restoring model weights from the end of the best epoch."
          print(log)
          #
          self.fn.write("%s\n" % log)
          self.fn.flush()
          #
          self.model.set_weights(self.best_weights) 
          self.best_model = self.model
          #
          self.model.save(os.path.join("SecondWave_%s" % country, "%s_SIR.h5" % country))
          #
          log = "Early Stopping!\nEpoch:%6i, Loss: %12.6f, N: %i, Best parameters: sigma: %.6f, sigma0: %.6f, Mu: %.6f, R0:%.6f" % ( self.stopped_epoch + 1, self.best, N, self.bestSigma, self.bestSigma0, self.bestM, self.bestSigma0/self.bestM )
          print(log)
          #
          self.fn.write("%s\n" % log)
          self.fn.flush()
          #
          self.plotBestState(epoch)
          #
          self.plotPredictions()
          #
          self.stop_training = True
          #
    self.step += 1
    #
    if not self.stop_training : 
      #
      log = "Epoch:%6i, N: %i, %s, S: %.6f, S0: %.6f, Mu: %.6f, R0:%.6f, P:%i" % (epoch + 1, N, " ".join("%s: %12.6f " % (k,v) for k,v in losses.items()), sigma.numpy(), sigma0.numpy(), mu.numpy(), sigma0.numpy()/mu.numpy(), self.wait)
      #
      print(log)
      #
      self.fn.write("%s\n" % log)
      self.fn.flush()
      #
      if self.step == 1 or self.step%(self.displayStep) == 0 : 
        self.plotBestState(epoch)
    #
    return self.stop_training


  def plotBestState(self, epoch) :
    #
    fig, ax1 = plt.subplots( figsize = (18, 8) )
    #
    plt.title("%s. $\sigma$: %.6f $\sigma_0$: %.6f $\mu$: %.6f $R_0$: %.6f" % (country, self.bestSigma, self.bestSigma0, self.bestM, self.bestSigma0/self.bestM))
    #
    color = 'tab:red'
    ax1.set_xlabel('Date')
    ax1.set_ylabel('I(t)', color = 'k')
    #
    # Original Data
    ax1.plot(self.tData, (N-N0)*self.yData + N0, 'go', markersize = 8, label = 'Reported data')
    #
    # Training Data
    ax1.plot(self.tTrainData, (N-N0)*np.exp(self.yTrainData) + N0, 'ro', markersize = 8, label = 'Training data')
    #
    # Predicted values at xValues PINN
    ax1.plot(self.tPredict, (N-N0)*np.exp(self.best_model.predict(self.tPredict)) + N0, color = "orange", label = 'PINN')
    #
    ax1.tick_params(axis = 'y', labelcolor = "k")
    plt.legend(loc = "upper left")
    
    ax1.set_xticks(tData[::7])
    ax1.set_xticklabels(dateTicks[::7])
    plt.setp(ax1.get_xticklabels(), rotation = 45, ha = "right", rotation_mode = "anchor")
    #
    ax1.grid()
    #
    ax2 = ax1.twinx()  # Instantiate a second axes that shares the same x-axis
    #
    # Plot a(t)
    color = 'tab:blue'
    ax2.set_ylabel('a(t)', color = 'b')
    #
    ax2.plot(self.tPredict, self.bestSigma*self.tPredict + self.bestSigma0 , "b-", label = "a(t)")
    ax2.plot(self.tPredict, [self.bestM]*len(self.tPredict), "k--", label = "$\mu$")
    #
    ax2.legend(loc = "upper right")
    ax2.tick_params(axis = 'y', labelcolor = 'b')
    #    
    fig.tight_layout()
    #
    plt.savefig(os.path.join("SecondWave_%s" % country, "%s_Best_%i.svg" % (country, epoch+1)),
                dpi = 300, orientation='landscape', format = 'svg')
    #
    plt.show()
    #
    plt.close()


  def plotPredictions(self) :
    #
    fig = plt.figure( figsize = (18, 8) )
    #
    plt.title("%s" % country)
    #
    color = 'tab:red'
    plt.xlabel('Date')
    plt.ylabel('I(t)', color = 'k')
    #
    # Original Data
    plt.plot(self.tData, (N-N0)*self.yData + N0, 'go', markersize = 8, label = 'Reported data')
    #
    # Predicted values during Training.
    plt.plot(self.tPredict, (N-N0)*np.exp(self.best_model.predict(self.tPredict)) + N0, color = "C1", marker = "o", markersize = 5)
    #
    # Short Term Predictions
    tShortPred = np.linspace(nOfDays, nOfDays + nOfShortPredDays - 1, nOfShortPredDays)
    plt.plot(tShortPred, (N-N0)*np.exp(self.best_model.predict(tShortPred))+ N0, color = 'C1', marker = "o", ls = "", fillstyle = 'none', markersize = 8, label = 'Short Term Predictions')
    #
    # Long Term Predictions
    tLongPred = np.linspace(nOfDays + nOfShortPredDays, nOfDays + nOfShortPredDays + nOfLongPredDays - 1, nOfLongPredDays)
    plt.plot(tLongPred, (N-N0)*np.exp(self.best_model.predict(tLongPred))+ N0, color = 'C1', ls = "--", label = 'Long Term Predictions')
    #
    if country == "Greece" :
      plt.vlines(x = 159, ymin = 0, ymax = 1.1*N, ls = "--", color = "r")
      plt.annotate("Lockdown", xy = (160,50), color = "r")
    #
    plt.grid()
    plt.legend(loc = "upper left")
    #
    tTicks = range(nOfDays + nOfShortPredDays + nOfLongPredDays)
    #
    plt.xticks(tTicks[::7], dateTicks[::7], rotation = 45, ha = "right", rotation_mode = "anchor")
    
    fig.tight_layout()
    #
    plt.savefig(os.path.join("SecondWave_%s" % country, "%s_Predictions.svg" % (country)),
                dpi = 300, orientation = 'landscape', format = 'svg')
    #
    plt.savefig(os.path.join("SecondWave_%s" % country, "%s_Predictions.jpeg" % (country)),
                dpi = 300, orientation = 'landscape', format = 'jpeg')
    #
    plt.show()
    #
    plt.close()

# Run the code for each one of the selected countries.

In [ ]:
for country, params in countries.items() :
  #
  if country in runOnly :
    #
    # Index of the first day of the second wave.
    t0 = params["SWt0"]
    #
    # Make a folder for the given country.
    if not os.path.exists("SecondWave_%s" % country) : os.mkdir("SecondWave_%s" % country)
    #
    # Country's data
    cntryOriginal = data[data["location"] == country]
    #
    # Keep only the dates after the first case.
    cntryCases = cntryOriginal[kind][cntryOriginal["total_%s" % kind.split("_")[1]] > 0].values.astype("float64")
    # Dates
    dates = cntryOriginal["date"][cntryOriginal["total_%s" % kind.split("_")[1]] > 0].values    
    #    
    # Fill gaps with ones.
    cntryCases[cntryCases < 1] = 1
    #
    # Keep only the second wave.
    cntryCases = cntryCases[t0:]
    dates = dates[t0:]
    #
    # Add the current day cases.
    #cntryCases = np.append(cntryCases, 916)
    #
    # Number of days for the second wave.
    nOfDays = len(cntryCases)
    #
    # Keep date zero ( One day before the first date with non-zero cases ).
    dateZero = datetime.strptime(dates[0], '%Y-%m-%d') - timedelta(days = 1)
    # and the last date.
    dateLast = str(datetime.strptime(dates[-1], '%Y-%m-%d') + timedelta(days = 1)).split()[0]
    #
    dateZero = str(dateZero).split()[0]
    #
    # Check for missing data and create Country's date labels.
    lastDateFound = False
    dateTicks = ["%s" % dateZero[5:]]
    #
    for d in range(1, nOfDays + nOfShortPredDays + nOfLongPredDays) :
      #
      nextDate = str(datetime.strptime(dateZero, '%Y-%m-%d') + timedelta(days = d)).split()[0]
      #
      if not lastDateFound :
        if nextDate not in dates :
          print ("Missing date", nextDate)
          cntryCases = np.insert(cntryCases, d, 0)
      #
      if nextDate == dates[-1] : lastDateFound = True
      #
      dateTicks.append("%s" % (nextDate[5:]))
    #
    # Time axis
    tData   = np.linspace(0, nOfDays-1, nOfDays)
    #
    # Smooth and normalize the data.
    #
    # Min Max scaler.
    N0, N = 0, max(cntryCases)
    #
    # Smoothing.
    ItDataOr = cntryCases
    #
    ItData = []
    if nOfDaysSmoothing > 0 : ItData.extend(ItDataOr[:nOfDaysSmoothing])
    #
    ItData.extend([sum(ItDataOr[i-nOfDaysSmoothing:i+nOfDaysSmoothing+1])/(2*nOfDaysSmoothing+1) for i in range(nOfDaysSmoothing,len(ItDataOr)-nOfDaysSmoothing)])
    #
    if nOfDaysSmoothing > 0 : ItData.extend(ItDataOr[-nOfDaysSmoothing:])
    #
    # Smoothed data.
    ItData = np.array(ItData)
    #
    # Normalize.
    ItData   = (ItData  -  N0)/(N-N0)
    ItDataOr = (ItDataOr - N0)/(N-N0)
    #
    # Plot the normalized data.
    fig, ax = plt.subplots( figsize = (18,6) )
    #
    plt.title(country)
    #
    # Plot the original data.
    ax.plot(tData, cntryCases, "ro")
    #
    ax.set_ylabel("I(t)")
    #
    # Plot the normalized and smoothed data.
    ax1 = ax.twinx()
    #
    ax1.plot(tData, ItDataOr, "ro", label = "Reported Data")
    ax1.plot(tData, ItData, "bo", fillstyle = "none", label = "Smoothed Data")
    #
    ax.grid()
    #
    ax1.set_xticks(tData[::7])
    ax1.set_xticklabels(dateTicks[::7])
    plt.setp(ax.get_xticklabels(), rotation = 45, ha = "right", rotation_mode = "anchor")
    #
    ax1.set_ylabel("Normalized I(t) (x %i)" % N)
    plt.xlabel("Date")
    #
    plt.legend()
    plt.show()
    plt.close()    
    #
    # Initialize the random generator seed.
    np.random.seed(45)
    tf.random.set_seed(45)
    #

    # Trainig data.
    # Data driven.
    xTrain    = tData.reshape(-1,1)
    xt = np.log(ItData)
    yTrain    = xt.reshape(-1,1)

    # PINN.
    tSIR  = np.linspace(0, nOfDays-1, nOfSIRPoints+1)
    xTrainSIR = tSIR.reshape(-1,1)

    # Initialize the PINN-SIR variables
    sigma  = tf.Variable([0.1], dtype = "float64")
    sigma0 = tf.Variable([0.1], dtype = "float64")
    #
    mu = tf.Variable([0.1], dtype = "float64")
    #
    # Use pre-trained model.
    if loadModel :
      #
      if os.path.exists( "SecondWave_%s/%s_SIR.h5" % (country, country) ) :
        model = tf.keras.models.load_model("SecondWave_%s/%s_SIR.h5" % (country, country) )
        print("Loading pre-trained country's model!")
      else :
        #
        model = tf.keras.models.load_model('sim_SIR.h5')
        print("Loading pre-trained SIR model!")
      #
    # Train a model from scratch.
    else :
      #
      # Xavier normal initializer.
      initializer = tf.keras.initializers.GlorotNormal()
      #
      act = "sigmoid"
      model = Sequential()
      model.add(Dense( 100, input_shape = (1,), activation = act, kernel_initializer = initializer) )
      model.add(Dense( 100, activation = act, kernel_initializer = initializer ))
      model.add(Dense( 100, activation = act, kernel_initializer = initializer ))
      model.add(Dense( 100, activation = act, kernel_initializer = initializer ))
      model.add(Dense( 100, activation = act, kernel_initializer = initializer ))
      #
      model.add(Dense(1))
      #

    # Model's summary.
    model.summary()
    #
    
    # Create an optimizer usinf Adam.
    optimizer = tf.keras.optimizers.Adam()

    # Creat a Logger object.
    logger = EarlyStopping(model = model, patience = patience, displayStep = displayStep, 
                           realData  = (tData, ItDataOr),
                           trainData = (tData, xt),
                           tPredict  = tSIR)

    # Store losses.
    losses = []

    # Custom training loop.
    for epoch in range(epochs): 
      #
      with tf.GradientTape(persistent = True) as tape:
        #
        # Data driven.
        yPred = model(tData.reshape(-1,1), training = True)
        #
        # Compute the MSE_{D} value.
        loss_valueMSE = loss_MSE(yTrain, yPred)
        #
        # Compute the MSE_{SIR}
        loss_valueSIR = loss_SIR()
        #
        # Compute the total Loss function.
        loss_value = loss_valueMSE + tf.reduce_mean(loss_valueSIR)
        
      # Retrieve the gradients of the trainable variables with respect to the loss.
      grads  = tape.gradient(loss_value, model.trainable_weights)
    
      # Apply the gradients.
      optimizer.apply_gradients(zip(grads, model.trainable_weights))
  
      # Retrieve the gradients of the trainable variables with respect to the loss_SIR.
      gradsA = tape.gradient(loss_valueSIR, [sigma, sigma0, mu])
      #
      # Apply the gradients.  
      optimizer.apply_gradients(zip(gradsA, [sigma, sigma0, mu]))
      #    
      losses.append( float(tf.reduce_mean(loss_value)) )
      #
      stop_training = logger.on_epoch_end(epoch, {"MSE_Data": float(tf.reduce_mean(loss_valueMSE)),
                                                  "MSE_SIR" : float(tf.reduce_mean(loss_valueSIR)), 
                                                  "TotLoss" : float(tf.reduce_mean(loss_value))})
      #
      if stop_training : break    